In [5]:
# Downloading preprocessed data, embeddings, models, and configs
!wget http://boytsov.info/datasets/knn4qa_demo2019-04-10.tar.bz2 

--2019-04-11 00:13:02--  http://boytsov.info/datasets/knn4qa_demo2019-04-10.tar.bz2
Resolving boytsov.info (boytsov.info)... 69.60.127.165
Connecting to boytsov.info (boytsov.info)|69.60.127.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45399656 (43M) [application/x-bzip2]
Saving to: ‘knn4qa_demo2019-04-10.tar.bz2’

knn4qa_demo2019-04- 100%[===================>]  43.30M  7.88MB/s    in 6.0s    

2019-04-11 00:13:08 (7.17 MB/s) - ‘knn4qa_demo2019-04-10.tar.bz2’ saved [45399656/45399656]



In [6]:
# Unpack the data 
!cd knn4qa ; tar jxvf ../knn4qa_demo2019-04-10.tar.bz2 

x WordEmbeddings/
x WordEmbeddings/squad/
x WordEmbeddings/squad/starspace/
x WordEmbeddings/squad/starspace/dim=100_squad_emb_unlemm_epoch1.answer
x WordEmbeddings/squad/starspace/dim=100_squad_emb_unlemm_epoch1.query
x lucene_index/
x lucene_index/squad/
x memfwdindex/
x memfwdindex/squad/
x misc/
x misc/cosine_fusion_dim=100_squad_emb_unlemm_epoch1.json
x misc/out_squad_train_20.model
x misc/bm25=text.json
x misc/oneFeat.model
x output/
x output/squad/
x output/squad/dev1/
x output/squad/test/
x output/squad/dev2/
x output/squad/train/
x output/squad/tran/
x output/squad/tran/qrels_all_graded.txt
x output/squad/tran/SolrQuestionFile.txt
x output/squad/tran/SolrAnswerFile.txt
x output/squad/tran/SolrAnswQuestFile.txt
x output/squad/train/qrels_all_graded.txt
x output/squad/train/SolrQuestionFile.txt
x output/squad/train/SolrAnswerFile.txt
x output/squad/train/SolrAnswQuestFile.txt
x output/squad/dev2/qrels_all_graded.txt
x output/squad/dev2/SolrQuestionFile.txt
x output/squad/dev2/So

In [8]:
# Create Lucene index
!cd knn4qa ; scripts/index/create_lucene_index.sh squad

Output directory: lucene_index/squad
Removing previous index (if exists)
MAVEN_OPTS=-Xms8388608k -Xmx14680064k -server
[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------------< edu.cmu.lti.oaqa:knn4qa >-----------------------
[INFO] Building Exploring if k-NN search can be useful for non-factoid QA. 2.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[WARNING] The POM for umass:RankLib:jar:2.7 is missing, no dependency information available
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ knn4qa ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 57 resources
[INFO] 
[INFO] --- maven-compiler-plugin:2.3.2:compile (default-compile) @ knn4qa ---
[INFO] Nothing to compile - all classes are up to date
[INFO] 
[INFO] --- exec-maven-plugin:1.3.2:java (default-cli) @ knn4qa ---
Creating a new Lucene index, maximum # of docs to process: 2147483647
Input file name: output/squad//train/Sol

In [11]:
# Create forward index
# First zero means bag-of-words only
# Second zero means we create a binary on-disk version of the index (slower to re-rank)
!cd knn4qa ; scripts/index/create_inmemfwd_index.sh squad 0 0

IN_DIR:   output/squad/
OUT_DIR:  memfwdindex/squad/
Storing word id seq param: 
In-mem index param param: 
Output directory: memfwdindex/squad/
MAVEN_OPTS=-Xms8388608k -Xmx14680064k -server
[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------------< edu.cmu.lti.oaqa:knn4qa >-----------------------
[INFO] Building Exploring if k-NN search can be useful for non-factoid QA. 2.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[WARNING] The POM for umass:RankLib:jar:2.7 is missing, no dependency information available
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ knn4qa ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 57 resources
[INFO] 
[INFO] --- maven-compiler-plugin:2.3.2:compile (default-compile) @ knn4qa ---
[INFO] Nothing to compile - all classes are up to date
[INFO] 
[INFO] --- exec-maven-plugin:1.3.2:java (default-cli) @ knn4qa ---
Processing field: 'text'
Use in-memory

In [16]:
# Export index to NMSLIB
!mkdir knn4qa/nmslib/squad
!cd knn4qa ; scripts/data/run_export_to_nmslib_sparse.sh \
-extr_json misc/cosine_fusion_dim=100_squad_emb_unlemm_epoch1.json \
-model_file  misc/out_squad_train_20.model  \
-memindex_dir memfwdindex/squad/ \
-embed_dir WordEmbeddings/squad/ \
-out_file nmslib/squad/cosine_fusion_dim=100_squad_emb_unlemm_epoch1.json_sparse.bin


MAVEN_OPTS=-Xms11184810k -Xmx14680064k -server
[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------------< edu.cmu.lti.oaqa:knn4qa >-----------------------
[INFO] Building Exploring if k-NN search can be useful for non-factoid QA. 2.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[WARNING] The POM for umass:RankLib:jar:2.7 is missing, no dependency information available
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ knn4qa ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 57 resources
[INFO] 
[INFO] --- maven-compiler-plugin:2.3.2:compile (default-compile) @ knn4qa ---
[INFO] Nothing to compile - all classes are up to date
[INFO] 
[INFO] --- exec-maven-plugin:1.3.2:java (default-cli) @ knn4qa ---
Finished loading context from file: memfwdindex/squad//text_unlemm
00:18:56.854 [edu.cmu.lti.oaqa.knn4qa.apps.ExportToNMSLIBSparse.main()] INFO  edu.cmu.lti.oaqa.knn4qa.embed.Embeddi

In [ ]:
# Finally starting the query server on port 8080 and creating the index using HNSW
!cd nmslib/query_server/cpp_client_server ; \
    ./query_server -i ../../../knn4qa/nmslib/squad/cosine_fusion_dim=100_squad_emb_unlemm_epoch1.json_sparse.bin \
    -s cosinesimil_sparse_bin_fast -m hnsw -c M=20,efConstruction=200 -t ef=200 -p 8080 

2019-04-11 00:57:47 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: dummy distance type: INT
2019-04-11 00:57:47 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: dummy distance type: FLOAT
2019-04-11 00:57:47 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: dummy distance type: DOUBLE
2019-04-11 00:57:47 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: bit_hamming distance type: INT
2019-04-11 00:57:47 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: leven distance type: INT
2019-04-11 00:57:47 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: normleven distance type: FLOAT
2019-04-11 00:57:47 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: kldivfast distance type: FLOAT
2019-04-11 00:57:47 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: kldivfast distance type: DOUBLE
2019-04-11 00:57:47 spacefactory.h:42 (Reg

2019-04-11 00:57:47 QueryService_server.cpp:125 (QueryServiceHandler) [INFO] Creating a new index copy
2019-04-11 00:57:47 hnsw.cc:180 (CreateIndex) [INFO] M                   = 20
2019-04-11 00:57:47 hnsw.cc:181 (CreateIndex) [INFO] indexThreadQty      = 8
2019-04-11 00:57:47 hnsw.cc:182 (CreateIndex) [INFO] efConstruction      = 200
2019-04-11 00:57:47 hnsw.cc:183 (CreateIndex) [INFO] maxM			          = 20
2019-04-11 00:57:47 hnsw.cc:184 (CreateIndex) [INFO] maxM0			          = 40
2019-04-11 00:57:47 hnsw.cc:186 (CreateIndex) [INFO] mult                = 0.333808
2019-04-11 00:57:47 hnsw.cc:187 (CreateIndex) [INFO] skip_optimized_index= 0
2019-04-11 00:57:47 hnsw.cc:188 (CreateIndex) [INFO] delaunay_type       = 2
2019-04-11 00:57:47 hnsw.cc:470 (SetQueryTimeParams) [INFO] Set HNSW query-time parameters:
2019-04-11 00:57:47 hnsw.cc:471 (SetQueryTimeParams) [INFO] ef(Search)         =20
2019-04-11 00:57:47 hnsw.cc:472 (SetQueryTimeParams) [INFO] algoType           =2

0%   10   20   3